In [1]:
from __future__ import print_function
import argparse
import os
import imp
import algorithms as alg
from dataloader import DataLoader, GenericDataset


exp = 'CIFAR10_ConvClassifier_on_RotNet_NIN4blocks_Conv2_feats_K20'
num_workers = 8
cuda = True
evaluate = False
checkpoint = 0
disp_step = 50
batch_size = 128
learning_rate = 0.1 
exp_name = 'RoNet_semiK20_lr0.1_bs128'

exp_config_file = os.path.join('.','config',exp+'.py')

# to adjust the learning rate and batch size, please go to config/CIFAR10_ConvClassifier_on_RotNet_NIN4blocks_Conv2_feats.py


# if args_opt.semi == -1:
exp_directory = os.path.join('.','experiments',exp)
# else:
#    assert(args_opt.semi>0)
#    exp_directory = os.path.join('.','experiments/unsupervised',args_opt.exp+'_semi'+str(args_opt.semi))

# Load the configuration params of the experiment
print('Launching experiment: %s' % exp_config_file)
config = imp.load_source("",exp_config_file).config
config['exp_dir'] = exp_directory # the place where logs, models, and other stuff will be stored
print("Loading experiment %s from file: %s" % (exp, exp_config_file))
print("Generated logs, snapshots, and model files will be stored on %s" % (config['exp_dir']))

# Set train and test datasets and the corresponding data loaders
data_train_opt = config['data_train_opt']
data_test_opt = config['data_test_opt']
num_imgs_per_cat = data_train_opt['num_imgs_per_cat'] if ('num_imgs_per_cat' in data_train_opt) else None



dataset_train = GenericDataset(
    dataset_name=data_train_opt['dataset_name'],
    split=data_train_opt['split'],
    random_sized_crop=data_train_opt['random_sized_crop'],
    num_imgs_per_cat=num_imgs_per_cat)
dataset_test = GenericDataset(
    dataset_name=data_test_opt['dataset_name'],
    split=data_test_opt['split'],
    random_sized_crop=data_test_opt['random_sized_crop'])

dloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=data_train_opt['batch_size'],
    unsupervised=data_train_opt['unsupervised'],
    epoch_size=data_train_opt['epoch_size'],
    num_workers=num_workers,
    shuffle=True)

dloader_test = DataLoader(
    dataset=dataset_test,
    batch_size=data_test_opt['batch_size'],
    unsupervised=data_test_opt['unsupervised'],
    epoch_size=data_test_opt['epoch_size'],
    num_workers=num_workers,
    shuffle=False)

config['disp_step'] = disp_step
algorithm = getattr(alg, config['algorithm_type'])(config)
if cuda: # enable cuda
    algorithm.load_to_gpu()
if checkpoint > 0: # load checkpoint
    algorithm.load_checkpoint(checkpoint, train= (not evaluate))

if not evaluate: # train the algorithm
    algorithm.solve(dloader_train, dloader_test, exp_name=exp_name)
else:
    algorithm.evaluate(dloader_test) # evaluate the algorithm


Launching experiment: ./config/CIFAR10_ConvClassifier_on_RotNet_NIN4blocks_Conv2_feats_K20.py
Loading experiment CIFAR10_ConvClassifier_on_RotNet_NIN4blocks_Conv2_feats_K20 from file: ./config/CIFAR10_ConvClassifier_on_RotNet_NIN4blocks_Conv2_feats_K20.py
Generated logs, snapshots, and model files will be stored on ./experiments/CIFAR10_ConvClassifier_on_RotNet_NIN4blocks_Conv2_feats_K20
Files already downloaded and verified
num_imgs_per_category 20
Files already downloaded and verified


2021-04-11 02:03:00,419 - algorithms.Algorithm - INFO   - Algorithm options {'data_train_opt': {'batch_size': 128, 'unsupervised': False, 'epoch_size': 50000, 'random_sized_crop': False, 'dataset_name': 'cifar10', 'split': 'train', 'num_imgs_per_cat': 20}, 'data_test_opt': {'batch_size': 128, 'unsupervised': False, 'epoch_size': None, 'random_sized_crop': False, 'dataset_name': 'cifar10', 'split': 'test'}, 'max_num_epochs': 100, 'out_feat_keys': ['conv2'], 'networks': {'feat_extractor': {'def_file': 'architectures/NetworkInNetwork.py', 'pretrained': './experiments/CIFAR10_RotNet_NIN4blocks/model_net_epoch200', 'opt': {'num_classes': 4, 'num_stages': 4, 'use_avg_on_conv3': False}, 'optim_params': None}, 'classifier': {'def_file': 'architectures/NonLinearClassifier.py', 'pretrained': None, 'opt': {'num_classes': 10, 'nChannels': 192, 'cls_type': 'NIN_ConvBlock3'}, 'optim_params': {'optim_type': 'sgd', 'lr': 0.1, 'momentum': 0.9, 'weight_decay': 0.0005, 'nesterov': True, 'LUT_lr': [(35, 0

AssertionError: 